In [ ]:
# Install the necessary Python packages for data processing and visualization.
!pip install pathway bokeh --quiet

In [ ]:
# Importing all important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
# Load the dataset into a DataFrame and display the few rows to understand its structure.
df=pd.read_csv("dataset.csv")
df

In [ ]:
df["SystemCodeNumber"].unique()

In [ ]:
# Encoding the required columns
df["VehicleType"]=df["VehicleType"].map({"car":2, "bike":1,"truck":3,"cycle":0})
df["TrafficConditionNearby"]=df["TrafficConditionNearby"].map({'low':0, 'high':2, 'average':1})
df["SystemCodeNumber"]=df["SystemCodeNumber"].map({'BHMBCCMKT01':0, 'BHMBCCTHL01':1, 'BHMEURBRD01':2, 'BHMMBMMBX01':3,'BHMNCPHST01':4, 'BHMNCPNST01':5, 'Broad Street':6, 'Others-CCCPS105a':7,
'Others-CCCPS119a':8, 'Others-CCCPS135a':9, 'Others-CCCPS202':10,'Others-CCCPS8':11, 'Others-CCCPS98':12, 'Shopping':13})

# Dropping columns which are not useful
df=df.drop("ID",axis="columns")

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
df

In [ ]:
# Create a file with only necessary columns i.e. occupancy, timestamp and capacity
df[["Timestamp", "Occupancy", "Capacity", "QueueLength","VehicleType","IsSpecialDay","TrafficConditionNearby",]].to_csv("parking_stream.csv", index=False)

In [ ]:
# Defining the schema for the columns
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    VehicleType: int
    IsSpecialDay: int
    TrafficConditionNearby: int

In [ ]:
# Loading data as a simulated stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# MODEL **1**

In [ ]:
# Make the pricing function
import datetime

delta_window1 = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(
        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# MODEL **2**

In [ ]:
# Make the pricing function
import datetime

delta_window2 = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occupancy_rate=pw.reducers.max(pw.this.Occupancy),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_weight=pw.reducers.max(pw.this.TrafficConditionNearby),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_weight=pw.reducers.max(pw.this.VehicleType),
    )
    .with_columns(
         demand = (3.0 * pw.this.occupancy_rate + 1.5 * pw.this.queue_length -
                 1.0 * pw.this.traffic_weight + 2.0 * pw.this.is_special_day +
                 1.0 * pw.this.vehicle_weight)
    )
    .with_columns(
        # Normalize demand
        normalized_demand = (pw.this.demand - (-1)) / (29 - (-1)),
    )
    .with_columns(
        # Calculate new price
        new_price = 10.0 * (1 + 0.3 * pw.this.normalized_demand)
    )
)

In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter1(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig


def price_plotter2(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "new_price", source=source, line_width=2, color="black")

    # Overlay green circles at each data point for better visibility
    fig.circle("t", "new_price", source=source, size=6, color="green")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz2 = delta_window2.plot(price_plotter2, sorting_col="t")
viz1 = delta_window1.plot(price_plotter1, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz1,viz2).servable()


In [ ]:
# Run the model

%%capture --no-display
pw.run()